# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119830e+02     1.390877e+00
 * time: 0.6925170421600342
     1     1.036793e+01     8.056206e-01
 * time: 2.066068172454834
     2    -1.126879e+01     9.749250e-01
 * time: 2.7046010494232178
     3    -3.376257e+01     7.192019e-01
 * time: 3.632336139678955
     4    -4.694520e+01     5.355576e-01
 * time: 4.574547052383423
     5    -5.695364e+01     1.973974e-01
 * time: 5.4482221603393555
     6    -5.990865e+01     1.339340e-01
 * time: 6.059966087341309
     7    -6.100932e+01     4.772501e-02
 * time: 6.690268039703369
     8    -6.144753e+01     4.967110e-02
 * time: 7.330810070037842
     9    -6.168352e+01     3.694102e-02
 * time: 7.95405912399292
    10    -6.189313e+01     2.904891e-02
 * time: 8.551769018173218
    11    -6.201647e+01     1.839874e-02
 * time: 9.154124975204468
    12    -6.206903e+01     1.568590e-02
 * time: 9.739629983901978
    13    -6.210778e+01     1.351465e-02
 * time: 10.340057134628296
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671054
    AtomicLocal         -18.8557627
    AtomicNonlocal      14.8522622
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485350 
    Xc                  -19.3336812

    total               -62.261666459692
